In [13]:
import re
import csv
import time
import nltk
import json
import string
import numpy as np
import pandas as pd
from nltk import stem
import sklearn.metrics
from random import randint
from numpy.linalg import norm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


with open('../new_userDict.json', 'r') as fp:
    
    new_userDict = json.load(fp)
    
    
with open('../new_isbnDict.json', 'r') as fp:
    
    new_isbnDict = json.load(fp)
    
    
with open('../dict_row.json', 'r') as fp:
    
    dict_row = json.load(fp)
    
    
with open('../dict_col.json', 'r') as fp:
    
    dict_col = json.load(fp)
    
print("Ok") 

Ok


In [14]:
# ATTENTION!!!  different respect to usersHaveRatedBook(s)

def usersHaveRatedBook(new_isbnDict, new_userDict, book_number, score):
    
    
    '''
    
    input:  new_isbnDict (Dict), new_userDict (Dict), book_number (int), score (int)
    
    action: select all users that have given a good rating to book_number
    
    output: users_rated (list)
    
    '''
    
    users_rated = []
    
    for user in new_isbnDict[str(book_number)]:
        
        if int(new_isbnDict[str(book_number)][user]) > score:
            
            try:
                
                new_userDict[user]
                users_rated.append(user)
                
            except:
                
                continue
            
    return list(set(users_rated))


def SimilarityUsers(utility_DataFrame, user_number, users_similar):
    
    '''
    
    input: utility_DataFrame (DataFrame), user_number (int), user_similar (List)
    
    action: compute cosine similarity between user_number and all the user in users_similar
    
    output: new_similarity (List of tuples)
    
    '''

    x = utility_DataFrame.loc[str(user_number)]
    x_length = norm(x)
    
    y = utility_DataFrame.loc[users_similar]
    y_length = norm(utility_DataFrame.loc[users_similar],axis=1)

    
    num = (y.values*x.values).sum(axis=1)
    den = x_length*y_length

    similarity = num/den
    similarity = np.nan_to_num(similarity)
    
    d = list(zip(list(users_similar),similarity))
    new_similarity = sorted(d, key=lambda tup: tup[1], reverse=True)
    
    
    return new_similarity

def ItemUsersRecommendation(new_similarity, new_userDict, k):
    
    '''
    
    input:  new_similarity(List of tuples), new_userDict(Dict), score(int), k(int)
    
    action: recommend items using the ratings of similar users
    
    output: recommendation (Dict), books (Dict)
    
    '''
    
    if new_similarity == []:
        
        return None
    
    if len(new_similarity) > k:
    
        recommendation = np.mean([u[1] for u in new_similarity[:k]]) 
    
    else:
        
        recommendation = np.mean([u[1] for u in new_similarity]) 
        
    return recommendation


def itemUsersScore(new_similarity,new_isbnDict, book_number, k):
    
    if new_similarity == []:
        
        return None
    
    if len(new_similarity) > k:
        
        score = [int(new_isbnDict[str(book_number)][u[0]]) for u in new_similarity[:k] if u[1] !=0.0]
        
    else:
        
        score = [int(new_isbnDict[str(book_number)][u[0]]) for u in new_similarity if u[1] !=0.0]
    
    return np.mean(score)


def CollaborativeFilteringItemUsersRMSE(utility_DataFrame, new_userDict, new_isbnDict, user_number, score_min, book_number, k):
    
    users_rated_book = usersHaveRatedBook(new_isbnDict, new_userDict, book_number, score_min)
    
    if users_rated_book == []:
        
        return None
    
    new_similarity = SimilarityUsers(utility_DataFrame, user_number, users_rated_book)
    
    
    if new_similarity == []:

        return None
    
    score = itemUsersScore(new_similarity,new_isbnDict, book_number, k)
    
    return score
    

In [15]:
def createSampleDict(new_isbnDict, new_userDict, t1, t2):
    

    small_isbnDict = {}

    for book in new_isbnDict:

        temp = new_isbnDict[book]
        i = 0

        for t in temp.values():
            if t != "0":
                i = i+1

        if i >t1:

            small_isbnDict[book] = new_isbnDict[book]   

    small_userDict = {}

    for user in new_userDict:

        temp = new_userDict[user]
        i = 0

        for t in temp.values():
            if t != "0":
                i = i+1

        if i >t2:

            small_userDict[user] = new_userDict[user]
            
            
    return small_isbnDict, small_userDict


def computeMatrices(train_userDict,train_isbnDict,small_userDict,small_isbnDict, dict_row, dict_col):

    n = len(small_isbnDict)
    m = len(small_userDict)
    
    index = sorted(small_userDict.keys())
    columns = sorted(small_isbnDict.keys())

    dict_row = {k:v for v,k in enumerate(index)}
    dict_col = {k:v for v,k in enumerate(columns)}

    u = np.zeros((m,n)) 
    R = np.zeros((m,n))
    for user in train_userDict:
        for isbn in train_userDict[user]:
            try:
                u[dict_row[user]][dict_col[isbn]] = train_userDict[user][isbn]
                R[dict_row[user]][dict_col[isbn]] = 1
            except:
                continue

    for isbn in train_isbnDict:
        for user in train_isbnDict[isbn]:
            try:
                u[dict_row[user]][dict_col[isbn]] = train_isbnDict[isbn][user]
                R[dict_row[user]][dict_col[isbn]] = 1
            except:
                continue
                
    small_utility_DataFrame = pd.DataFrame(u, index = index, columns = columns)
    R = pd.DataFrame(R, index = index, columns = columns)

    return u, R, small_utility_DataFrame


def selectSample(i,b,list_user,small_userDict, small_isbnDict):

    test_index = [k for k in range((i*b),(i+1)*b)]
    train_index = [k for k in range(len(list_user)) if k not in test_index]

    train = list_user[train_index]
    test  = list_user[test_index]

    train_userDict = {}
    train_isbnDict = {isbn:{} for isbn in small_isbnDict}

    for user in train:

        train_userDict[user] = small_userDict[user]

    for isbn in small_isbnDict:
        for user in small_isbnDict[isbn]:

            try:
                train_userDict[user]
                train_isbnDict[isbn][user] = small_isbnDict[isbn][user]
            except:
                continue
                
                
    test_userDict = {}
    test_isbnDict = {isbn:{} for isbn in small_isbnDict}

    for user in test:

        test_userDict[user] = small_userDict[user]

    for isbn in small_isbnDict:
        for user in small_isbnDict[isbn]:

            try:
                test_userDict[user]
                test_isbnDict[isbn][user] = small_isbnDict[isbn][user]
            except:
                continue
                
    return train_userDict, train_isbnDict, test_userDict, test_isbnDict, test


#indices = [i for i in range(len(list_user))]
#indices = np.array(indices)
#k = 5
#b = len(small_userDict)//k


def selectSampleRandom(indices,b,list_user,small_userDict, small_isbnDict):
    
    test_index = np.random.choice(indices, size = b, replace=False)
    
    indices = np.delete(indices,test_index)
    
    train_index = [k for k in range(len(list_user)) if k not in test_index]
    

    train = list_user[train_index]
    test  = list_user[test_index]

    train_userDict = {}
    train_isbnDict = {isbn:{} for isbn in small_isbnDict}

    for user in train:

        train_userDict[user] = small_userDict[user]

    for isbn in small_isbnDict:
        for user in small_isbnDict[isbn]:

            try:
                train_userDict[user]
                train_isbnDict[isbn][user] = small_isbnDict[isbn][user]
            except:
                continue
                
    return train_userDict, train_isbnDict, test_userDict, test_isbnDict, test, indices

In [16]:
def mainItemUsersRMSE(utility_DataFrame, R, new_userDict, new_isbnDict, score_min=0, k=3):
    

    rmse_dict = {j:{} for j in new_userDict.keys()}

    for user in new_userDict:

        for book in new_userDict[user]:

            if new_userDict[user][book] != "0":

                try:
                    new_isbnDict[book]
                    rmse_dict[user][book] = new_userDict[user][book]
                except:

                    continue

    d = rmse_dict.keys()
    v = []

    for key in d:

        if rmse_dict[key] == {}:

            v.append(key)

    for i in v:

        del rmse_dict[i]

    rmse_vector_itemUsers = []
    
    #i = 0
    
    for user in rmse_dict:
        
        #i = i+1
        #if i%1000 == 0:
            
        #    print("Pause")
        #    time.sleep(30)

        for isbn in rmse_dict[user]:
            
            prediction_score = None
            user_number = user
            book_number = isbn
            
            try:
                prediction_score = CollaborativeFilteringItemUsersRMSE(utility_DataFrame, new_userDict, new_isbnDict, 
                                                                   user_number, score_min, book_number, k)
                
                #print(prediction_score)
                true_score = utility_DataFrame[book_number][user_number]
                
                if prediction_score == None:
                    
                    term1 = utility_DataFrame[book_number]
                    r1 = R[book_number]
                    term1 = term1[r1 ==1]
                    
                    term2 = utility_DataFrame.loc[user_number]
                    r2 = R.loc[user_number]
                    term2 = term2[r2==1]
                    
                    prediction_score = (np.mean(term1) + np.mean(term2))/2
            
            except:
                continue

            if prediction_score != None:

                rmse_vector_itemUsers.append(tuple([user_number, book_number, prediction_score, true_score]))


    rmse = np.sqrt((np.array([u[2]-u[3] for u in rmse_vector_itemUsers])**2).sum()/len(rmse_vector_itemUsers))
    
    return rmse, rmse_vector_itemUsers

In [5]:
u, R, small_utility_DataFrame = computeMatrices(small_userDict,small_isbnDict,small_userDict,small_isbnDict,dict_row,dict_col)

rmse_test, rmse_vector_itemUsers = mainItemUsersRMSE(small_utility_DataFrame,R,small_userDict,small_isbnDict,score_min=0,k=3)

In [ ]:
rmse_vector_itemUsers[:100]

# K-fold cross validation for collaborative filtering user-based

In [17]:
def itemUsers_KfoldsCV(small_userDict,small_isbnDict, dict_row, dict_col, kk):

    RMSE = []
    #kk = 5
    list_user = np.array(sorted(small_userDict.keys()))
    b = len(small_userDict)//kk

    for iterator in range(kk):

        train_userDict, train_isbnDict, test_userDict, test_isbnDict, test =  selectSample(iterator,b,list_user,
                                                                                           small_userDict, small_isbnDict)

        u, R, small_utility_DataFrame = computeMatrices(train_userDict,small_isbnDict,small_userDict,small_isbnDict,dict_row,dict_col)

        rmse_test, rmse_vector_itemUsers = mainItemUsersRMSE(small_utility_DataFrame,R,test_userDict,test_isbnDict,score_min=0,k=3)
        print("Ok")

        RMSE.append(rmse_test)
        
    return RMSE

In [18]:
small_userDict = {k:new_userDict[k] for k in list(new_userDict.keys())[:5000]}
small_isbnDict = {k:new_isbnDict[k] for k in list(new_isbnDict.keys())[:5000] }

In [ ]:
small_isbnDict, small_userDict = createSampleDict(new_isbnDict, new_userDict, 20, 20)

In [19]:
RMSE = itemUsers_KfoldsCV(new_userDict,new_isbnDict, dict_row, dict_col, 5)

Ok
Ok
Ok
Ok
Ok


In [20]:
print("rmse collaborative filtering user-based: ",np.mean(RMSE))

rmse collaborative filtering user-based:  1.19140763006
